In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from pathlib import Path

2023-10-20 12:50:27.693399: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("train*.tfrecords"))

In [14]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['age']
    return (ecg_data, example['gender']), label

@tf.function
def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

@tf.function
def age_lt_90(x,y):
    return tf.math.reduce_all(tf.math.less_equal(y, tf.constant([90.0])))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    dataset = dataset.filter(age_lt_90)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [15]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [16]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

In [6]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [20]:
def residual_unit(x, y, n_samples_out, n_filters_out, prefix, kernel_size = 16):
    n_samples_in = y.shape[1]
    downsample = n_samples_in // n_samples_out
    n_filters_in = y.shape[2]
    
    if downsample == 1:
        y = y
    else:
        y = layers.MaxPooling1D(downsample, strides=downsample, padding='same', name = f"{prefix}_mp_opt")(y)
        
    if n_filters_in != n_filters_out:
        y = layers.Conv1D(n_filters_out, 1, padding='same', use_bias=False, name = f"{prefix}_conv_opt")(y)
        
    x = layers.Conv1D(n_filters_out, kernel_size, padding='same', use_bias=False, name = f"{prefix}_conv1")(x)
    x = layers.BatchNormalization(name = f"{prefix}_bn1")(x)
    x = layers.Activation("relu", name = f"{prefix}_act1")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout1")(x)
    x = layers.Conv1D(n_filters_out, kernel_size, strides=downsample, padding='same', use_bias=False, name = f"{prefix}_conv2")(x)

    x = layers.Add(name = f"{prefix}_add")([x,y])
    y = x
    x = layers.BatchNormalization(name = f"{prefix}_bn2")(x)
    x = layers.Activation("relu", name = f"{prefix}_act2")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout2")(x)
    return (x,y)
    
ecg_input_layer = tf.keras.layers.Input(shape=(5000,12), name="ecg_input")
age_input = tf.keras.layers.Input(shape=(1,), name="age_input")
x = layers.Conv1D(64, 16, padding='same', use_bias=False, name = "conv_1")(ecg_input_layer)
x = layers.BatchNormalization(name="bn")(x)
x = layers.Activation("relu", name="relu")(x)

x, y = residual_unit(x,x,1024,128, "res1")
x, y = residual_unit(x,y,256,196, "res2")
x, y = residual_unit(x,y,64,256, "res3")
x, _ = residual_unit(x,y,16,320, "res4")

x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Add(name="add")([x, age_input])
x = tf.keras.layers.Dense(1, name="output")(x)

model = tf.keras.models.Model([ecg_input_layer, age_input], x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 ecg_input (InputLayer)      [(None, 5000, 12)]           0         []                            
                                                                                                  
 conv_1 (Conv1D)             (None, 5000, 64)             12288     ['ecg_input[0][0]']           
                                                                                                  
 bn (BatchNormalization)     (None, 5000, 64)             256       ['conv_1[0][0]']              
                                                                                                  
 relu (Activation)           (None, 5000, 64)             0         ['bn[0][0]']                  
                                                                                            

In [22]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "resnet-age"))
]

model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10
   7479/Unknown - 517s 68ms/step - loss: 132.6500 - mse: 132.6500 - mae: 9.0489

2023-10-20 13:12:32.855599: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14541092026103740123
2023-10-20 13:12:32.861908: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 12414231050322713288
2023-10-20 13:12:32.861934: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16180197146222263964
2023-10-20 13:14:39.281965: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 15053203262210092414
2023-10-20 13:14:39.287488: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 4340312151314173204
2023-10-20 13:14:39.287520: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10110123543950772168


INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


7479/7479 [==============================] - 648s 86ms/step - loss: 132.6500 - mse: 132.6500 - mae: 9.0489 - val_loss: 166.9662 - val_mse: 166.9662 - val_mae: 10.1294 - lr: 0.0010
Epoch 2/10
7479/7479 [==============================] - ETA: 0s - loss: 110.9823 - mse: 110.9823 - mae: 8.3008INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


...

7479/7479 [==============================] - 641s 86ms/step - loss: 79.6919 - mse: 79.6919 - mae: 7.0364 - val_loss: 99.2426 - val_mse: 99.2426 - val_mae: 7.9275 - lr: 0.0010
Epoch 9/10
7479/7479 [==============================] - ETA: 0s - loss: 75.8964 - mse: 75.8964 - mae: 6.8676INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


7479/7479 [==============================] - 642s 86ms/step - loss: 75.8964 - mse: 75.8964 - mae: 6.8676 - val_loss: 87.6635 - val_mse: 87.6635 - val_mae: 7.4072 - lr: 0.0010
Epoch 10/10
7479/7479 [==============================] - ETA: 0s - loss: 71.8023 - mse: 71.8023 - mae: 6.6899INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


INFO:tensorflow:Assets written to: data/models/resnet-age-2023-10-20_13-03-55/assets


7479/7479 [==============================] - 641s 86ms/step - loss: 71.8023 - mse: 71.8023 - mae: 6.6899 - val_loss: 89.7037 - val_mse: 89.7037 - val_mae: 7.4743 - lr: 0.0010


In [23]:
model.save("data/models/resnet-age2.keras")